In [18]:
import sys
sys.path.append('..')
import flickrapi
import nationalparks.secrets as secrets

In [23]:
flickr = flickrapi.FlickrAPI(secrets.api_key, secrets.api_secret, format='parsed-json')

In [25]:
sets   = flickr.photosets.getList(user_id='73509078@N00')
title  = sets['photosets']['photoset'][0]['title']['_content']
print('First set title: %s' % title)

First set title: En Route


In [26]:
sets

{'photosets': {'page': 1,
  'pages': 1,
  'perpage': 500,
  'total': 110,
  'photoset': [{'id': '72157674639812906',
    'owner': '73509078@N00',
    'username': 'Sybren A. Stüvel',
    'primary': '30014551651',
    'secret': 'cfdd173178',
    'server': '8742',
    'farm': 9,
    'count_views': '48',
    'count_comments': '0',
    'count_photos': 7,
    'count_videos': 0,
    'title': {'_content': 'En Route'},
    'description': {'_content': 'Photos I take en route to work.'},
    'can_comment': 0,
    'date_create': '1475529863',
    'date_update': '1475530523',
    'photos': 7,
    'videos': 0,
    'visibility_can_see_set': 1,
    'needs_interstitial': 0},
   {'id': '72157654330832908',
    'owner': '73509078@N00',
    'username': 'Sybren A. Stüvel',
    'primary': '18931589641',
    'secret': '532382be61',
    'server': '507',
    'farm': 1,
    'count_views': '42',
    'count_comments': '0',
    'count_photos': 26,
    'count_videos': 0,
    'title': {'_content': 'Hasselblad 150mm 

In [28]:
flickr = flickrapi.FlickrAPI(secrets.api_key, secrets.api_secret, format='json')
raw_json = flickr.photosets.getList(user_id='73509078@N00')
# raw_json -> '{...}'

import json
parsed = json.loads(raw_json.decode('utf-8'))
parsed

{'photosets': {'page': 1,
  'pages': 1,
  'perpage': 500,
  'total': 110,
  'photoset': [{'id': '72157674639812906',
    'owner': '73509078@N00',
    'username': 'Sybren A. Stüvel',
    'primary': '30014551651',
    'secret': 'cfdd173178',
    'server': '8742',
    'farm': 9,
    'count_views': '48',
    'count_comments': '0',
    'count_photos': 7,
    'count_videos': 0,
    'title': {'_content': 'En Route'},
    'description': {'_content': 'Photos I take en route to work.'},
    'can_comment': 0,
    'date_create': '1475529863',
    'date_update': '1475530523',
    'photos': 7,
    'videos': 0,
    'visibility_can_see_set': 1,
    'needs_interstitial': 0},
   {'id': '72157654330832908',
    'owner': '73509078@N00',
    'username': 'Sybren A. Stüvel',
    'primary': '18931589641',
    'secret': '532382be61',
    'server': '507',
    'farm': 1,
    'count_views': '42',
    'count_comments': '0',
    'count_photos': 26,
    'count_videos': 0,
    'title': {'_content': 'Hasselblad 150mm 